In [1]:
import cv2 as cv
import numpy as np
from time import sleep
import os
os.chdir('../../data')

In [5]:
cap = cv.VideoCapture('road.mp4')
BGS = cv.BackgroundSubtractorMOG2()
while cap.isOpened():
    _, frame = cap.read()